### Данный Jupiter-ноутбук предназначен для запуска на Jupyter-сервере внутри AI Cloud. Если вы хотите выполнить те же самые действия из удаленного Jupyter-сервера, посмотрите пример отправки задач на обучение через Training Job API.

# 0. Подключение библиотеки для работы с кластером и сервисом

In [1]:
try:
    import client_lib
except ImportError:
    raise RuntimeError("Скрипт не предназначен для запуска вне кластера")

In [ ]:
import requests

In [ ]:
def save_file(url, filename):
    # Download file and place it on local storage
    r = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f"{filename} downloaded from {url}")

# 1. Размещаем модель и данные на локальные диски кластера

In [ ]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_tf2/mnist.npz", "mnist.npz")
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_tf2/requirements.txt", "requirements.txt")
save_file("https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/job_launch_tf2/tensorflow_mnist_estimator.py", "tensorflow_mnist_estimator.py")

# 2. Сборка кастомного образа с нужными библиотеками

##### Содержимое файла requirements.txt

In [2]:
%cat /home/jovyan/quick-start/job_launch_tf2/requirements.txt

# tensorflow-gpu 2.3.0 already installed in registry.aicloud.sbcp.ru/base/horovod-cuda10.1-tf2.3.0
# with other libraries, we add one more:
requests

##### Запуск сборки кастомного образа с необходимыми библиотеками

In [3]:
job = client_lib.ImageBuildJob(
                    from_image='registry.aicloud.sbcp.ru/base/horovod-cuda10.1-tf2.3.0', # базовый образ
                    requirements_file='/home/jovyan/quick-start/job_launch_tf2/requirements.txt' # файл с зависимостями для кастомного образа
)

job.submit()

'ImageBuildJob "{\'image\': \'registry.aicloud.sbcp.ru/ai0000001-0254/b5594f73-fd8b-4224-ba7c-81101499e073\', \'name\': \'image-build-job-q29xh\', \'status\': \'ok\'}" created'

In [4]:
job.new_image # идентификатор кастомного образа

'registry.aicloud.sbcp.ru/ai0000001-0254/b5594f73-fd8b-4224-ba7c-81101499e073'

In [5]:
job.logs() # просмотр логов сборки образа в интерактивном режиме

"Job=image-build-job-q29xh is not ready yet, try again in 10 seconds"


# 3. Запуск задачи обучения на кластере

#### Создание задачи обучения и отправка ее на кластер

Параметр `base_image` предназначен для запуска задачи на кластере в своем кастомном образе (см. пункт 2 выше). Переменная `job.new_image` содержит название собранного кастомного образа (вида: `registry.aicloud.sbcp.ru/ai0000001-0254/b5594f73-fd8b-4224-ba7c-81101499e073`).

По окончании сборки кастомного образа его можно указать в поле `base_image` и отправить задачу обучения на кластер с использованием этого образа:
```python
mnist_tf_run = client_lib.Job(base_image=job.new_image, # кастомный образ
                              script='/home/jovyan/quick-start/job_launch_tf2/tensorflow_mnist_estimator.py',
                              n_workers=2, n_gpus=4, warm_cache=False)
```

In [6]:
# base_image='registry.aicloud.sbcp.ru/base/horovod-cuda10.1-tf2.3.0' - базовый образ, можно заменить на свой (смотри выше)
mnist_tf_run = client_lib.Job(base_image='registry.aicloud.sbcp.ru/base/horovod-cuda10.1-tf2.3.0',
                              script='/home/jovyan/quick-start/job_launch_tf2/tensorflow_mnist_estimator.py',
                              n_workers=2, n_gpus=4, warm_cache=False)

В предыдущей строке мы указали образ, в рамках которого будет исполняться задача (`base_image`), скрипт, который будет запущен (`script`), а также количество рабочих узлов кластера (`n_workers`) и количество GPU-карт на каждом (`n_gpus`). В этом случае под задачу будет запущено 2 рабочих узла по 4 GPU карты (итого — 8 GPU).

Теперь вызовем на объекте метод `.submit()` и отправим задачу в очередь исполнения на кластере.

In [7]:
mnist_tf_run.submit()

'Job "lm-mpi-job-f02e6f73-8ba4-4e26-a078-b4a29b615bc4" created'

#### Мониторинг задач

Вы можете просмотреть список ваших задач и их статус с помощью следующего метода:

In [ ]:
client_lib.jobs()

Логи задачи:

In [ ]:
mnist_tf_run.logs()

Логи по названию задачи:

In [ ]:
client_lib.logs("lm-mpi-job-244763c4-e1d1-44c0-8558-a03fefba26f6")

(ЕСЛИ НЕОБХОДИМО ПРЕРВАТЬ ЗАДАЧУ) Варианты остановки задачи:

In [ ]:
mnist_tf_run.kill()

In [ ]:
client_lib.kill('lm-mpi-job-244763c4-e1d1-44c0-8558-a03fefba26f6')

# 4. Сохранение промежуточных результатов обучения модели

Если в процессе обучения модели пользователь сохраняет промежуточные результаты (checkpoints) обучения, они попадают в папку `/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/`. Их можно скачать через веб-интерфейс Jupyter-ноутбука или скопировать из локально доступной файловой системы в хранилище S3. Внутри кода задачи обучения можно сохранять метрики модели с помощью `mlflow` (пример — в коде `tensorflow_mnist_estimator.py`).

## Выгрузка результатов обучения модели с NFS на S3

##### Указываем параметры доступа к бакету S3

Для перемещения данных из локального хранилища `/home/jovyan` в объектное хранилище S3 необходимо указать параметры доступа к бакету S3. Эти данные можно найти на портале portal.sbercloud.ru в параметрах заказанной услуги AI Cloud ("Мои услуги"->"AI Cloud").

В одинарных кавычках `''` введите  по порядку: `S3 namespace`, `S3 access key`, `S3 security key`

In [ ]:
# Specify S3 credentials as follows
# save_aws_credentials('b175178b-f537-4a1c-8977-7871bb9dc448-namespace', 'b13f3480-a701-48fb-85c5-b4a3b36f3713', 'BPBdF5E9CGAFv+rmBakFc+R0krVCHd/u4rPsqfG8')
client_lib.save_aws_credentials('', '', '')

Ниже укажем бакет s3, в который будут копироваться файлы

In [ ]:
# Specify S3 bucket
# s3_bucket = "s3://9b8d0e96-6461-47cf-9507-21ede838c9ac-bucket/"
s3_bucket = ""

In [ ]:
# Перемещение отдельных файлов
# client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/checkpoint.data-00000-of-00001", s3_bucket).submit()
# client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/checkpoint.ckpt.index", s3_bucket).submit()
# client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/checkpoint", s3_bucket).submit()

In [ ]:
# Перемещение папки с файлами
client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/", s3_bucket, recursive=True).submit()

Если все было сделано правильно, то промежуточные результаты обучения модели сохранятся в объектном хранилище S3, из которого можно будет впоследствии восстановить объект с моделью